In [1]:
pip install faiss-cpu langchain langchain-community langchain-groq transformers sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
# Define product knowledge base content
products_data = {
    "products_1.txt": """Product: SmartSpeaker X100
Features:
- Voice Assistant Enabled
- 360-degree sound
- Bluetooth & WiFi connectivity

FAQs:
Q: How do I reset my SmartSpeaker?
A: Hold the power button for 10 seconds until the LED flashes.

Troubleshooting:
- If the speaker is unresponsive, try unplugging and replugging it.
- Ensure your WiFi is stable before attempting a firmware update.
""",

    "products_2.txt": """Product: UltraBook Pro 15
Features:
- 15.6-inch Retina Display
- 16GB RAM, 1TB SSD
- AI-Powered Battery Optimization

FAQs:
Q: How do I extend battery life?
A: Reduce screen brightness and close background apps.

Troubleshooting:
- If the laptop does not turn on, hold the power button for 30 seconds.
- Check if the charging cable is properly connected.
""",

    "products_3.txt": """Product: Fitness Tracker Z
Features:
- Heart Rate Monitoring
- Sleep Tracking
- Water Resistant (IP68)

FAQs:
Q: Can I swim with my Fitness Tracker Z?
A: Yes, it is water-resistant up to 1.5 meters.

Troubleshooting:
- If the tracker isn’t syncing, restart Bluetooth and reconnect.
- Ensure your app is updated to the latest version.
"""
}

# Create and write content to files
for filename, content in products_data.items():
    with open(filename, "w", encoding="utf-8") as file:
        file.write(content)
    print(f"Created: {filename}")

print("All product knowledge base files have been created successfully!")

Created: products_1.txt
Created: products_2.txt
Created: products_3.txt
All product knowledge base files have been created successfully!


In [3]:
import os
import getpass
import faiss
import json
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model

# Securely set API key for Groq
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

# Initialize the Groq chat model
llm = init_chat_model("llama3-8b-8192", model_provider="groq")

# Load multiple product knowledge bases
def load_knowledge_bases():
    knowledge_files = ["products_1.txt", "products_2.txt", "products_3.txt"]
    all_docs = []

    for file in knowledge_files:
        if not os.path.exists(file):
            print(f"Warning: Knowledge base '{file}' not found. Skipping...")
            continue
        loader = TextLoader(file)
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        split_docs = text_splitter.split_documents(docs)
        all_docs.extend(split_docs)

    if not all_docs:
        raise ValueError("No valid knowledge bases found. Please check the file paths.")

    return all_docs

# Create FAISS vector store
def create_vector_store(docs):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore

# Load and vectorize documents
try:
    documents = load_knowledge_bases()
    vector_db = create_vector_store(documents)
except ValueError as e:
    print(f"Error: {e}")
    exit(1)

# Memory for conversation retention
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Specialized Tool 1: FAQ Retrieval
def get_faq_answer(user_query):
    retrieved_docs = vector_db.similarity_search(user_query, k=1)
    if not retrieved_docs:
        return "No relevant FAQ found."

    return retrieved_docs[0].page_content.strip()

# Specialized Tool 2: Troubleshooting Support
def troubleshoot_issue(user_query):
    retrieved_docs = vector_db.similarity_search(user_query, k=3)
    if not retrieved_docs:
        return "No troubleshooting steps found for this issue."

    context = " ".join([doc.page_content for doc in retrieved_docs])
    prompt = f"User Issue: {user_query}\nContext: {context}\n\nProvide step-by-step troubleshooting instructions:"

    response = llm.invoke(prompt)
    return response.content.strip()

# Main Customer Support Function
def customer_support(user_query):
    if not user_query.strip():
        return json.dumps({"error": "Empty query. Please ask a valid question."}, indent=4)

    try:
        retrieved_docs = vector_db.similarity_search(user_query, k=3)
        context = " ".join([doc.page_content for doc in retrieved_docs])

        prompt = f"User Query: {user_query}\nContext: {context}\n\nProvide a well-structured response:"
        response = llm.invoke(prompt)

        structured_response = {
            "query": user_query,
            "context": context[:500], \
            "response": response.content.strip(),
            "faq_answer": get_faq_answer(user_query),
            "troubleshooting": troubleshoot_issue(user_query)
        }
        return json.dumps(structured_response, indent=4)

    except Exception as e:
        return json.dumps({"error": f"An error occurred: {str(e)}"}, indent=4)

# Example Usage
if __name__ == "__main__":
    user_input = input("Ask a support question: ")
    print(customer_support(user_input)) # How do I connect my SmartSpeaker X100 to WiFi?


Enter API key for Groq: ··········


<ipython-input-3-c13094cfa047>:41: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-3-c13094cfa047>:54: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Ask a support question: How do I connect my SmartSpeaker X100 to WiFi?
{
    "query": "How do I connect my SmartSpeaker X100 to WiFi?",
    "context": "Product: SmartSpeaker X100\nFeatures:\n- Voice Assistant Enabled\n- 360-degree sound\n- Bluetooth & WiFi connectivity\n\nFAQs:\nQ: How do I reset my SmartSpeaker?\nA: Hold the power button for 10 seconds until the LED flashes.\n\nTroubleshooting:\n- If the speaker is unresponsive, try unplugging and replugging it.\n- Ensure your WiFi is stable before attempting a firmware update. Product: Fitness Tracker Z\nFeatures:\n- Heart Rate Monitoring\n- Sleep Tracking\n- Water Resistant (IP68)\n\nFAQs:\nQ: Can I swim with ",
    "response": "**Connecting Your SmartSpeaker X100 to WiFi**\n\nThe SmartSpeaker X100 is a sleek and powerful speaker that comes with both Bluetooth and WiFi connectivity. With its voice assistant enabled, you can easily stream your favorite music, podcasts, or audiobooks from the cloud or your phone/tablet. Here's a step-